In [1]:
import os
import sys
import yaml
import dask
import zarr
import numpy as np
import xarray as xr
import pandas as pd
from glob import glob

import calendar
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

sys.path.insert(0, os.path.realpath('../libs/'))
import verif_utils as vu

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

In [74]:
base_dir_mlevel = '/glade/derecho/scratch/ksha/CREDIT_data/ERA5_mlevel_1deg/'
base_dir_plevel = '/glade/derecho/scratch/ksha/CREDIT_data/ERA5_plevel_1deg/'
base_dir_output = base_dir_mlevel

In [75]:
fn_fmt_static = base_dir_mlevel + 'static/ERA5_mlevel_1deg_static_subset.zarr'

In [76]:
ds_static = xr.open_zarr(fn_fmt_static)
coef_a = ds_static['coef_a'].values
coef_b = ds_static['coef_b'].values
coef_a_mid = 0.5*(coef_a[1:] + coef_a[:-1])
coef_b_mid = 0.5*(coef_b[1:] + coef_b[:-1])

In [77]:
ds_static['coef_a_mid'] = xr.DataArray(
    coef_a_mid,
    dims=('level'),
    coords={
        'level': ds_static['level']
    }
)

ds_static['coef_b_mid'] = xr.DataArray(
    coef_b_mid,
    dims=('level'),
    coords={
        'level': ds_static['level']
    }
)

In [81]:
ds_static['level'] = np.arange(18).astype(np.float32)

In [83]:
save_name = base_dir_mlevel + 'static/ERA5_mlevel_1deg_static_for_interp.zarr'

In [85]:
# ds_static.to_zarr(save_name, mode='w')

In [87]:
# xr.open_zarr(save_name)

In [72]:
ds_out = xr.open_dataset('/glade/derecho/scratch/ksha/CREDIT/RAW_OUTPUT/fuxi_mlevel_dry/2021-01-01T00Z/pred_2021-01-01T00Z_012.nc')

In [73]:
ds_out['level']

<xarray.DataArray 'level' (level: 18)>
array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17])
Coordinates:
  * level    (level) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17

In [69]:
nc_ds = xr.open_dataset('/glade/u/home/ksha/miles-physics/credit/metadata/ERA5_Lev_Info.nc')

In [70]:
nc_ds

<xarray.Dataset>
Dimensions:     (half_level: 138, level: 137, latitude: 640, longitude: 1280)
Coordinates:
  * half_level  (half_level) int32 1 2 3 4 5 6 7 ... 132 133 134 135 136 137 138
  * latitude    (latitude) float64 89.78 89.51 89.23 ... -89.23 -89.51 -89.78
  * level       (level) int32 1 2 3 4 5 6 7 8 ... 131 132 133 134 135 136 137
  * longitude   (longitude) float64 0.0 0.2812 0.5625 ... 359.2 359.4 359.7
    time        datetime64[ns] ...
Data variables:
    a_half      (half_level) float64 ...
    a_model     (level) float64 ...
    b_half      (half_level) float64 ...
    b_model     (level) float64 ...
    utc_date    int32 ...
    weight      (latitude) float64 ...
    zero        (latitude) float64 ...
Attributes: (12/18)
    RDA_DATASET:                            ds633.6
    RDA_DATASET_URL:                        https:/rda.ucar.edu/datasets/ds63...
    RDA_DATASET_DOI:                        DOI: 10.5065/XV5R-5344
    RDA_DATASET_GROUP:                      ERA5 atmospheric model level anal...
    DATA_SOURCE:                            Downloaded by NCAR/CISL/DECS from...
    DOWNLOAD_SPECIALIST:                    Chi-Fan Shih, NCAR/CISL/DECS, chi...
    ...                                     ...
    NETCDF_COMPRESSION:                     Precision-preserving compression ...
    CONVERSION_AND_COMPRESSION_PLATFORM:    cheyenne node employed: Linux r8i...
    CONVERSION_AND_COMPRSSION_DATE:         Mon Nov 21 16:44:28 MST 2022
    CONVERSION_AND_COMPRESSION_SPECIALIST:  Dave Stepaniak, NCAR/CISL/DECS, d...
    history:                                Mon Nov 21 16:44:49 2022: ncks -4...
    NCO:                                    netCDF Operators version 5.0.3 (H...

In [71]:
nc_ds['level']

<xarray.DataArray 'level' (level: 137)>
array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,  14,
        15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,
        29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,
        43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,
        57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,
        71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,  84,
        85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,  98,
        99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112,
       113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126,
       127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137], dtype=int32)
Coordinates:
  * level    (level) int32 1 2 3 4 5 6 7 8 9 ... 130 131 132 133 134 135 136 137
    time     datetime64[ns] ...
Attributes:
    long_name:      model level
    standard_name:  atmosphere_hybrid_sigma_pressure_coordinate
    short_name:     lev
    formula:        p = a + b*ps
    order:          near top of atmosphere to near surface
    units:          1